In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from io import BytesIO
from zipfile import ZipFile
import urllib.request
import os

In [2]:
browser = webdriver.Chrome('C:/Users/jamie/Bootcamp/Homework/Homework_Assignment_20_Tableau/Resources/chromedriver')

In [3]:
browser.get('https://s3.amazonaws.com/tripdata/index.html')

In [4]:
#get all links
all_links = [link.get_attribute('href') for link in browser.find_elements_by_tag_name('a')]

In [5]:
# extract necessary links
linksNY_18 = [link for link in all_links if 'JC' not in link if '2018' in link]

In [6]:
# function for downloading, unziping and dataframing data from a link
def create_df(link):
    
    url = urllib.request.urlopen(link)
    output = open('temporary.zip', 'wb')    
    output.write(url.read())
    output.close()
    dataframe = pd.read_csv('temporary.zip')
    
    if (len(dataframe.columns) == 15):
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID', 'User Type',
       'Birth_Year', 'Gender']
    else:
        dataframe.columns = ['Trip Duration (sec)', 'Start Time', 'Stop Time', 'Start Station ID',
       'Start Station Name', 'Start Station Latitude',
       'Start Station Longitude', 'End Station ID', 'End Station Name',
       'End Station Latitude', 'End Station Longitude', 'Bike ID',
       'Localized Value', 'User Type', 'Birth_Year', 'Gender']
        dataframe = dataframe.drop('Localized Value',1)
        
    print(link)
    os.remove('temporary.zip')
    
    return dataframe

In [7]:
# function for cleaning and preparing df
def prepare_df(df):
    
#     drop n/a
    df = df.dropna(how='any').reset_index(drop=True)
    
#     change data types
#    df['Birth_Year'] = df.Birth_Year.astype(int)
    df['Start Time'] = pd.to_datetime(df['Start Time'])
    df['Stop Time'] = pd.to_datetime(df['Stop Time'])
    
    return df

In [8]:
# create empty df
NY18 = pd.DataFrame()

In [9]:
for link in linksNY_18:
    temporary_df = create_df(link)
    NY18 = NY18.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/201801-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201802-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201803-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201804-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201805-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201806-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201807-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201808-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201809-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201810-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201811-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/201812-citibike-tripdata.csv.zip


In [10]:
# clean and prepare df
NY18 = prepare_df(NY18)

In [11]:
NY18.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth_Year,Gender
0,970,2018-01-01 13:50:57.434,2018-01-01 14:07:08.186,72.0,W 52 St & 11 Ave,40.767272,-73.993929,505.0,6 Ave & W 33 St,40.749013,-73.988484,31956,Subscriber,1992,1
1,723,2018-01-01 15:33:30.182,2018-01-01 15:45:33.341,72.0,W 52 St & 11 Ave,40.767272,-73.993929,3255.0,8 Ave & W 31 St,40.750585,-73.994685,32536,Subscriber,1969,1
2,496,2018-01-01 15:39:18.337,2018-01-01 15:47:35.172,72.0,W 52 St & 11 Ave,40.767272,-73.993929,525.0,W 34 St & 11 Ave,40.755942,-74.002116,16069,Subscriber,1956,1
3,306,2018-01-01 15:40:13.372,2018-01-01 15:45:20.191,72.0,W 52 St & 11 Ave,40.767272,-73.993929,447.0,8 Ave & W 52 St,40.763707,-73.985162,31781,Subscriber,1974,1
4,306,2018-01-01 18:14:51.568,2018-01-01 18:19:57.642,72.0,W 52 St & 11 Ave,40.767272,-73.993929,3356.0,Amsterdam Ave & W 66 St,40.774667,-73.984706,30319,Subscriber,1992,1


In [12]:
NY18.dtypes

Trip Duration (sec)                 int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                  float64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                    float64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth_Year                          int64
Gender                              int64
dtype: object

In [13]:
NY18.count()

Trip Duration (sec)        17545842
Start Time                 17545842
Stop Time                  17545842
Start Station ID           17545842
Start Station Name         17545842
Start Station Latitude     17545842
Start Station Longitude    17545842
End Station ID             17545842
End Station Name           17545842
End Station Latitude       17545842
End Station Longitude      17545842
Bike ID                    17545842
User Type                  17545842
Birth_Year                 17545842
Gender                     17545842
dtype: int64

In [16]:
NY18.to_csv('Data/NYC_2018_citibike_from_url.csv', encoding='utf-8', index=False)

In [17]:
linksJC_18 = [link for link in all_links if 'JC' in link if '2018' in link]

In [18]:
# create empty df
JC18 = pd.DataFrame()

In [19]:
for link in linksJC_18:
    temporary_df = create_df(link)
    JC18 = JC18.append(temporary_df, ignore_index=True, sort=False)

https://s3.amazonaws.com/tripdata/JC-201801-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201802-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201803-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201804-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201805-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201806-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201807-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201808-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201809-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201810-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201811-citibike-tripdata.csv.zip
https://s3.amazonaws.com/tripdata/JC-201812-citibike-tripdata.csv.zip


In [20]:
# clean and prepare df
JC18 = prepare_df(JC18)

In [21]:
JC18.head()

,Trip Duration (sec),Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth_Year,Gender
0,932,2018-01-01 02:06:17.541,2018-01-01 02:21:50.027,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31929,Subscriber,1992,1
1,550,2018-01-01 12:06:18.039,2018-01-01 12:15:28.443,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31845,Subscriber,1969,2
2,510,2018-01-01 12:06:56.978,2018-01-01 12:15:27.810,3183,Exchange Place,40.716247,-74.033459,3199,Newport Pkwy,40.728745,-74.032108,31708,Subscriber,1946,1
3,354,2018-01-01 14:53:10.186,2018-01-01 14:59:05.096,3183,Exchange Place,40.716247,-74.033459,3267,Morris Canal,40.712419,-74.038526,31697,Subscriber,1994,1
4,250,2018-01-01 17:34:30.192,2018-01-01 17:38:40.984,3183,Exchange Place,40.716247,-74.033459,3639,Harborside,40.719252,-74.034234,31861,Subscriber,1991,1


In [22]:
JC18.dtypes

Trip Duration (sec)                 int64
Start Time                 datetime64[ns]
Stop Time                  datetime64[ns]
Start Station ID                    int64
Start Station Name                 object
Start Station Latitude            float64
Start Station Longitude           float64
End Station ID                      int64
End Station Name                   object
End Station Latitude              float64
End Station Longitude             float64
Bike ID                             int64
User Type                          object
Birth_Year                          int64
Gender                              int64
dtype: object

In [23]:
JC18.count()

Trip Duration (sec)        353892
Start Time                 353892
Stop Time                  353892
Start Station ID           353892
Start Station Name         353892
Start Station Latitude     353892
Start Station Longitude    353892
End Station ID             353892
End Station Name           353892
End Station Latitude       353892
End Station Longitude      353892
Bike ID                    353892
User Type                  353892
Birth_Year                 353892
Gender                     353892
dtype: int64

In [24]:
JC18.to_csv('Data/JC_2018_citibike_from_url.csv', encoding='utf-8', index=False)

In [25]:
NYdf = NY18.append(JC18, ignore_index=True, sort=False)

In [26]:
NYdf.to_csv('Data/NYdf_2018_citibike_from_url.csv', encoding='utf-8', index=False)